# Recomendador de perfis
---
Uma vez computado o índice contendos os K vizinhos mais próximos de cada um dos 85.000+ usuários e a lista de atribuição dos perfis, um recomendador de perfis será construído baseado em:

1 - A vizinhança de um usuário

2 - Os perfis que a vizinhança possui

3 - Os perfis que o usuário não possui

4 - As chamadas atribuições categóricas dos perfis ainda não atribuídos ao usuário

In [1]:
import os
import numpy as np
import pandas as pd
import pynndescent
import duckdb
import pickle
import scipy.sparse
from knn_indexer import KnnIndex

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

As variáveis de ambiente abaixo precisam ser configuradas antes da execução deste notebook. Vide o arquivo **setenv.ps1.example**

In [3]:
DATASET                = os.environ['DATASET']
USER_ROLES             = os.environ['USER_ROLES']
ORGUNIT_ROLES          = os.environ['ORGUNIT_ROLES']
FUNCTION_ROLES         = os.environ['FUNCTION_ROLES']
KNN_INDEX              = os.environ['KNN_INDEX']
HASHED_FEATURES_IDX    = os.environ['HASHED_FEATURES_IDX']
CUTOFF_DISTANCE        = 1.0

Leitura do arquivo contendos os dados do usuário e os perfis já atribuídos

In [4]:
dataset_df = pd.read_parquet(DATASET)
dataset_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91798 entries, 0 to 91797
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   chave_usuario            91798 non-null  object
 1   tipo_usuario             91798 non-null  object
 2   centro_custo             39913 non-null  object
 3   lotacao_topo             91798 non-null  object
 4   sigla_lotacao            91798 non-null  object
 5   nome_lotacao             91798 non-null  object
 6   cargo                    91798 non-null  object
 7   enfase                   91798 non-null  object
 8   funcao                   91798 non-null  object
 9   sindicato                39415 non-null  object
 10  area_rh                  91264 non-null  object
 11  imovel                   91264 non-null  object
 12  local_negocio            91261 non-null  object
 13  grupo_prestacao_servico  51118 non-null  object
 14  regime_trabalho          76638 non-nul

In [5]:
user_roles_df = pd.read_parquet(USER_ROLES)
user_roles_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413991 entries, 0 to 413990
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   chave_usuario  413991 non-null  object
 1   role           413991 non-null  object
dtypes: object(2)
memory usage: 6.3+ MB


Leitura do arquivo contendo as atribuições dos perfis as diferentes lotações

In [6]:
orgunit_roles_df = pd.read_parquet(ORGUNIT_ROLES)
orgunit_roles_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28190 entries, 0 to 28189
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   lotacao_topo   28190 non-null  object
 1   sigla_lotacao  28190 non-null  object
 2   role           28190 non-null  object
 3   atribuicoes    28190 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 881.1+ KB


Leitura do arquivo contendo as atriabuições do perfil para os tipos de usuário, cargos, ênfases e funções gratificadas

In [7]:
function_roles_df = pd.read_parquet(FUNCTION_ROLES)
function_roles_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14013 entries, 0 to 14012
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tipo_usuario  14013 non-null  object
 1   cargo         14013 non-null  object
 2   enfase        14013 non-null  object
 3   funcao        14013 non-null  object
 4   role          14013 non-null  object
 5   atribuicoes   14013 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 657.0+ KB


Leitura do índice pré-computado de distâncias entre os 85.000+ usuários

In [8]:
knn_index = KnnIndex.load(KNN_INDEX)
knn_index.get_neighbors(0)

(array([    0, 29301, 16115, 85550, 27715, 69840, 29939, 29008, 25392,
        19955, 40571, 61978,   958,  8853, 27035, 55804, 11492, 26039,
         5356, 63071, 61360,  5624, 62911, 10523, 27483,  5663, 20132,
        27484, 30088,  3673, 25557, 68230, 14074, 63010, 57121, 26533,
        26093, 14721, 62846, 28929, 26792, 35310, 70013,  1609, 34924,
        31611, 85531, 26134, 76791, 26430, 58558, 28586, 64330, 34070,
        35295,  5596, 87270,  9960, 30490, 35230, 34123, 28583,  8680,
        33236,  1021,  5862, 35024, 14868, 63679, 28390, 64022, 19596,
        34034, 68663,  1207, 62974, 18846, 34923,  1707, 25323, 35710,
        24243, 58823, 40446,  2222, 26534, 30285,  2241, 27355,  2235,
        26416, 13147, 26990, 85494, 58697, 40488, 63805, 18435, 73850,
        11969, 32633,  2229, 87377, 26193, 26927, 39901, 76662, 40579,
         2923, 70699,  3185, 16682, 18450,  2937, 32675, 26857, 90735,
        32641, 78922, 31347, 32624, 70475, 28617,  2223, 26635, 16132,
      

Leitura do índice contendo os dados dos usuários para uso da matriz esparsa de features por usuário

In [9]:
hashed_features_idx = pd.read_parquet(HASHED_FEATURES_IDX)
hashed_features_idx.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91798 entries, 0 to 91797
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          91798 non-null  int64 
 1   chave_usuario  91798 non-null  object
 2   lotacao_topo   91798 non-null  object
 3   sigla_lotacao  91798 non-null  object
 4   tipo_usuario   91798 non-null  object
 5   cargo          91798 non-null  object
 6   enfase         91798 non-null  object
 7   funcao         91798 non-null  object
dtypes: int64(1), object(7)
memory usage: 5.6+ MB


O trecho de código abaixo mostra como realizar uma busca no índice de usuários para recuperar a posição em que um usuário específico situa-se na matriz esparsa numérica e com isso recuperar sua vizinhança bem como as distâncias calculadas de seus vizinhos.

Um DataFrame é construído possibilitando a exibição destes dados e seu uso em conjunto com a biblioteca **DuckDB**

In [10]:
search_user   = "C2Y6"
search_df     = hashed_features_idx[hashed_features_idx["chave_usuario"] == search_user].copy()

print(f"Usuário '{search_user}' encontrado na massa de perfis analisada")
search_index   = search_df.iloc[0]["index"]
indices, distances = knn_index.get_neighbors(search_index)
_neighbors_df = pd.DataFrame({"index": indices, "distance": distances})

display(search_df)
display(_neighbors_df)

Usuário 'C2Y6' encontrado na massa de perfis analisada


,index,chave_usuario,lotacao_topo,sigla_lotacao,tipo_usuario,cargo,enfase,funcao
15417,15417,C2Y6,SI,SI/GIPD/GIA,EMPREGADO,PROF. PETROBRAS DE NIVEL SUPERIOR PLENO,PCR NS ANALISE DE SISTEMAS PROC NEGOCIOS,n/a


,index,distance
0,15417,0.000000
1,25099,45.382812
2,8755,46.300808
3,24878,46.550179
4,87088,48.249760
5,20326,48.285210
6,22247,48.431767
7,65182,48.586819
8,82499,48.608089
9,81881,48.812069


A query abaixo é utilizada para retornar a vizinhança de um usuário e, para cada vizinho, a listagem de perfis atribuídos à cada vizinho que ainda não foram atribuídos ao usuário.

As chamadas atribuições categóricas são também computadas para as roles dos vizinhos.


 - atribuido_tipo_usuario - O perfil possui atribuição ao mesmo tipo de usuário
 
 - atribuido_topo - O perfil possui atribuilção à mesma lotação topo do usuário
 
 - atribuido_lotacao - O perfil possui atribuição à mesma lotação do usuário
 
 - atribuido_cargo - O perfil possuí atribuição ao mesmo cargo do usuário
 
 - atribuido_enfase - O perfil possui atribuição à mesma ênfase do usuário
 
 - atribuido_funcao - O perfil possui atribuição à mesma função gratificada do usuário
 
 
Em todos os casos acima, as atribuições categóricas dizem respeito à perfis ainda não atribuídos ao próprio usuário. Estas informações figuram de forma proeminente no cômputo do score da recomendação conforme abaixo.

A consulta é executada 50 vezes para que qualquer regressão de performance na query seja imediatamente observada.

In [11]:
conn = duckdb.connect(":memory:")

In [12]:
sql = """
WITH cte_atribuido_tipo_usuario AS (
    SELECT DISTINCT tipo_usuario, role FROM function_roles_df
)
, cte_atribuido_topo AS (
    SELECT DISTINCT lotacao_topo, role FROM orgunit_roles_df
)
, cte_atribuido_lotacao AS (
    SELECT DISTINCT sigla_lotacao, role FROM orgunit_roles_df
)
, cte_atribuido_cargo AS (
    SELECT DISTINCT cargo, role FROM function_roles_df
)
, cte_atribuido_enfase AS (
    SELECT DISTINCT enfase, role FROM function_roles_df
)
, cte_atribuido_funcao AS (
    SELECT DISTINCT funcao, role FROM function_roles_df
)
SELECT  DISTINCT z.chave_usuario as usuario_busca
,       a.distance
,       b.chave_usuario
,       b.sigla_lotacao
,       b.cargo
,       b.enfase
,       b.funcao
,       c.role
,       CASE WHEN e.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_tipo_usuario
,       CASE WHEN f.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_topo        
,       CASE WHEN g.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_lotacao
,       CASE WHEN h.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_cargo
,       CASE WHEN i.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_enfase
,       CASE WHEN j.role IS NULL THEN 0 ELSE 1 END                   AS atribuido_funcao
FROM    search_df z
        --
        CROSS JOIN _neighbors_df a
        --
        INNER JOIN hashed_features_idx b
        ON a.index              = b.index
        --
        INNER JOIN user_roles_df c
        ON b.chave_usuario      = c.chave_usuario
        --
        LEFT OUTER JOIN user_roles_df d
        ON  z.chave_usuario     = d.chave_usuario
        AND c.role              = d.role
        --
        LEFT OUTER JOIN cte_atribuido_tipo_usuario e
        ON  z.tipo_usuario      = e.tipo_usuario 
        AND c.role              = e.role
        --
        LEFT OUTER JOIN cte_atribuido_topo f
        ON  z.lotacao_topo      = f.lotacao_topo 
        AND c.role              = f.role
        --
        LEFT OUTER JOIN cte_atribuido_lotacao g
        ON  z.sigla_lotacao     = g.sigla_lotacao 
        AND c.role              = g.role
        --
        LEFT OUTER JOIN cte_atribuido_cargo h
        ON  z.cargo             = h.cargo 
        AND c.role              = h.role
        --
        LEFT OUTER JOIN cte_atribuido_enfase i
        ON  z.enfase            = i.enfase 
        AND c.role              = i.role
        --
        LEFT OUTER JOIN cte_atribuido_funcao j
        ON  z.funcao            = j.funcao 
        AND c.role              = j.role
        --
WHERE   b.chave_usuario         <> z.chave_usuario
AND     z.chave_usuario         IS NOT NULL
AND     d.role                  IS NULL
AND     f.role                  IS NOT NULL -- atribuido_topo        
ORDER   BY a.distance
"""
for i in range(50): # to catch performance regressions
    neighbors_df = conn.execute(sql).fetchdf()
neighbors_df

,usuario_busca,distance,chave_usuario,sigla_lotacao,cargo,enfase,funcao,role,atribuido_tipo_usuario,atribuido_topo,atribuido_lotacao,atribuido_cargo,atribuido_enfase,atribuido_funcao
0,C2Y6,46.300808,BCH4,SI/GIPD,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ENGENHARIA DE PRODUCAO,n/a,Z:MM_PBAUT_FISCAL_CONTRATO,1,1,0,1,1,1
1,C2Y6,46.550179,CYDC,SI/GIPD/GIA,PROF. PETROBRAS DE NIVEL SUPERIOR MASTER,PCR NS ANALISE DE SISTEMAS INFRAESTRUTURA,n/a,Z:BC_PE_BASIS_SEG_PERFIS,0,1,1,0,0,1
2,C2Y6,49.309269,U3ZI,AUDITORIA/ANSEF/ADP,PROF. PETROBRAS DE NIVEL SUPERIOR PLENO,PCR NS ENGENHARIA DE PRODUCAO,n/a,Z:MM_PB001_CONSULTANTE_INF_MES,1,1,0,1,1,1
3,C2Y6,49.962723,EDYQ,SI/GIPD/GIA,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ANALISE DE SISTEMAS PROC NEGOCIOS,n/a,Z:BC_PE_BASIS_SEG_PERFIS,0,1,1,0,0,1
4,C2Y6,50.226982,UP06,TIC/CORP/FRIE,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS ANALISE DE SISTEMAS PROC NEGOCIOS,n/a,Z:MM_PBAUT_FISCAL_CONTRATO,1,1,0,1,1,1
5,C2Y6,53.357643,A097,SI/GIPD,PROF. PETROBRAS DE NIVEL SUPERIOR MASTER,PCR NS ANALISE DE SISTEMAS PROC NEGOCIOS,n/a,Z:MM_PBAUT_FISCAL_CONTRATO,1,1,0,1,1,1
6,C2Y6,54.022770,XZ53,TIC/CORP/FRIE/PN-CTRI,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS CIENCIAS CONTABEIS,n/a,Z:MM_PBAUT_FISCAL_CONTRATO,1,1,0,1,1,1
7,C2Y6,54.635887,U3X3,AUDITORIA/ANSEF/ADOW,PROF. PETROBRAS DE NIVEL SUPERIOR PLENO,PCR NS CIENCIAS CONTABEIS,n/a,Z:MM_PB001_CONSULTANTE_INF_MES,1,1,0,1,1,1
8,C2Y6,56.160141,CYM3,SI/GRC,PROF. PETROBRAS DE NIVEL SUPERIOR MASTER,PCR NS ANALISE DE SISTEMAS PROC NEGOCIOS,n/a,Z:MM_PBAUT_FISCAL_CONTRATO,1,1,0,1,1,1
9,C2Y6,56.631783,CJE2,AUDITORIA/ANSEF/ADOW,PROF. PETROBRAS DE NIVEL SUPERIOR SENIOR,PCR NS CIENCIAS CONTABEIS,n/a,Z:MM_PB001_CONSULTANTE_INF_MES,1,1,0,1,1,1


A query abaixo não é utilizada pelo recomendador mas ilustra as distâncias e quantitativos de usuários e perfis por gerências

In [13]:
sql = f"""
    SELECT  a.sigla_lotacao
    ,       MIN(distance)                   AS min_distance
    ,       AVG(distance)                   AS avg_distance
    ,       MAX(distance)                   AS max_distance
    ,       COUNT(DISTINCT a.chave_usuario) AS qtd_usuarios
    ,       COUNT(DISTINCT a.role)          AS qtd_roles
    FROM    neighbors_df a
    GROUP   BY a.sigla_lotacao
    --
    UNION
    --
    SELECT  '<todas lotações>'              AS sigla_lotacao
    ,       MIN(distance)                   AS min_distance
    ,       AVG(distance)                   AS avg_distance
    ,       MAX(distance)                   AS max_distance
    ,       COUNT(DISTINCT a.chave_usuario) AS qtd_usuarios
    ,       COUNT(DISTINCT a.role) AS qtd_roles
    FROM    neighbors_df a
    --
    ORDER   BY qtd_roles DESC
"""
for i in range(50): # to catch performance regressions
    lotacoes_vizinhas_df = conn.execute(sql).fetchdf()
lotacoes_vizinhas_df

,sigla_lotacao,min_distance,avg_distance,max_distance,qtd_usuarios,qtd_roles
0,<todas lotações>,46.300808,51.715818,56.631783,10,3
1,SI/GIPD,46.300808,49.829226,53.357643,2,1
2,SI/GIPD/GIA,46.550179,48.256451,49.962723,2,1
3,AUDITORIA/ANSEF/ADP,49.309269,49.309269,49.309269,1,1
4,TIC/CORP/FRIE,50.226982,50.226982,50.226982,1,1
5,TIC/CORP/FRIE/PN-CTRI,54.022770,54.022770,54.022770,1,1
6,AUDITORIA/ANSEF/ADOW,54.635887,55.633835,56.631783,2,1
7,SI/GRC,56.160141,56.160141,56.160141,1,1


A query abaixo retorna os perfis recomendados com seus respectivos scores calculados.

**TODO: Inserir notação matemática da fórmula do score aqui como um imagem**

Query executada 50 vezes para capturar possíveis regressões de performance

In [14]:
sql = """
WITH cte_roles AS (
    SELECT  a.role
    ,       MAX(atribuido_tipo_usuario)     AS atribuido_tipo_usuario
    ,       MAX(atribuido_topo)             AS atribuido_topo
    ,       MAX(atribuido_lotacao)          AS atribuido_lotacao
    ,       MAX(atribuido_cargo)            AS atribuido_cargo
    ,       MAX(atribuido_enfase)           AS atribuido_enfase
    ,       MAX(atribuido_funcao)           AS atribuido_funcao
    ,       MAX(atribuido_tipo_usuario)
    +       MAX(atribuido_topo)
    +       MAX(atribuido_lotacao)
    +       MAX(atribuido_cargo)
    +       MAX(atribuido_enfase)
    +       MAX(atribuido_funcao)           AS atribuicoes_categoricas
    ,       MIN(distance)                   AS min_distance
    ,       AVG(distance)                   AS avg_distance
    ,       MAX(distance)                   AS max_distance    
    ,       (1.0 
            -   (
                    (
                        MIN(distance) / MAX(distance) 
                    +   AVG(distance) / MAX(distance)
                    ) 
                *   0.5 
                )
            )                               AS distance_factor
    ,       COUNT()                         AS qtd_atribuicoes
    FROM    neighbors_df a
    WHERE   a.role <> 'Z:BC_USO_GERAL'
    GROUP   BY a.role
)
, cte_role_scores AS (
    SELECT  a.role
    ,      ( 
               a.atribuicoes_categoricas 
           *   a.qtd_atribuicoes 
           *   a.distance_factor 
           )
           /    
           ( 
               (SELECT MAX(atribuicoes_categoricas) FROM cte_roles) 
           *   (SELECT MAX(qtd_atribuicoes)         FROM cte_roles)
           *   (SELECT MAX(distance_factor)         FROM cte_roles)
           ) AS score   
    FROM   cte_roles a
)
SELECT  a.role
,       a.atribuicoes_categoricas
,       a.min_distance
,       a.avg_distance
,       a.max_distance
,       a.distance_factor
,       a.qtd_atribuicoes
,       b.score
,       a.atribuido_tipo_usuario
,       a.atribuido_topo
,       a.atribuido_lotacao
,       a.atribuido_cargo
,       a.atribuido_enfase
,       a.atribuido_funcao
FROM    cte_roles a
        --
        INNER JOIN cte_role_scores b
        ON  a.role                      = b.role
        --
WHERE   b.score                         > 0.0
ORDER   BY b.score DESC
"""

for i in range(50): # to catch performance regressions
    recomendations_df = conn.execute(sql).fetchdf()
recomendations_df

,role,atribuicoes_categoricas,min_distance,avg_distance,max_distance,distance_factor,qtd_atribuicoes,score,atribuido_tipo_usuario,atribuido_topo,atribuido_lotacao,atribuido_cargo,atribuido_enfase,atribuido_funcao
0,Z:MM_PBAUT_FISCAL_CONTRATO,5,46.300808,52.013669,56.160141,0.124695,5,1.000000,1,1,0,1,1,1
1,Z:MM_PB001_CONSULTANTE_INF_MES,5,49.309269,53.525646,56.631783,0.092074,3,0.443036,1,1,0,1,1,1
2,Z:BC_PE_BASIS_SEG_PERFIS,3,46.550179,48.256451,49.962723,0.051226,2,0.098595,0,1,1,0,0,1
